<a href="https://colab.research.google.com/github/atzingan/DAEN690/blob/main/Forecasting%20Models/TotalHuntersbySpecies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

#File stored in GitHub LFS
url = 'https://github.com/gjrsas/DAEN690/raw/main/Data/vw_harvest_estimates.csv'

df = pd.read_csv(url)
print(df.shape)

(9797, 23)


In [ ]:
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error, mean_absolute_error
def load_data(url):
    return pd.read_csv(url)

def preprocess_data(data):
    data['season'] = pd.to_datetime(data['season'], format='%Y')
    data.sort_values('season', inplace=True)
    return data

def fit_arima_model(train_series, order=(3, 1, 3)):
    arima_model = ARIMA(train_series.astype(np.float64), order=order)
    return arima_model.fit()

def calculate_metrics(actual, predicted):
    rmse = np.sqrt(mean_squared_error(actual, predicted))
    mae = mean_absolute_error(actual, predicted)
    mape = np.mean(np.abs((actual - predicted) / actual)) * 100
    return rmse, mae, mape

def main():
    data = load_data(url)

    # Group and sum the data by species only
    grouped_data = data.groupby(['sp_group_estimated', 'season'])['active_hunters'].sum().reset_index()

    # Get unique species
    species_list = grouped_data['sp_group_estimated'].unique()

    # Create a DataFrame to store the results
    results_df = pd.DataFrame(columns=['Species', 'Prediction', 'RMSE', 'MAE', 'MAPE'])

    # Loop through each species
    for species in species_list:
        specific_data = grouped_data[(grouped_data['sp_group_estimated'] == species)]
        specific_data = preprocess_data(specific_data)

        if len(specific_data) > 5:  # Require at least 6 data points to split
            train_data = specific_data.iloc[:-2]  # Leave out the last 2 points for testing
            test_data = specific_data.iloc[-2:]

            time_series_train = train_data.set_index('season')['active_hunters']
            time_series_test = test_data.set_index('season')['active_hunters']

            try:
                # Fit ARIMA model on training set
                arima_results = fit_arima_model(time_series_train)

                # Predict on the test set
                predictions = arima_results.predict(start=time_series_test.index[0], end=time_series_test.index[-1])
                rmse, mae, mape = calculate_metrics(time_series_test, predictions)

                # Forecast the next year beyond the available data
                forecast = arima_results.get_forecast(steps=1).predicted_mean.iloc[0]

                # Append the results
                results_df = results_df.append({
                    'Species': species,
                    'Prediction': forecast,
                    'RMSE': rmse,
                    'MAE': mae,
                    'MAPE': mape
                }, ignore_index=True)

            except Exception as e:
                print(f"Could not fit ARIMA model for Species: {species}. Error: {e}")

    # Optionally, save the results to an Excel file
    results_df.to_excel('arima_forecast_results_by_species.xlsx', index=False)

if __name__ == "__main__":
    main()

<ipython-input-2-ddeb5afae191>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['season'] = pd.to_datetime(data['season'], format='%Y')
<ipython-input-2-ddeb5afae191>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.sort_values('season', inplace=True)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No freq